Import Libraries

In [1]:

import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np
from keras.optimizers import Adam
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten

  Label Creation


In [ ]:
classes=[]
filename=r'D:\images_part1'
for sub_folder in os.listdir(os.path.join(filename,"train")):
    classes.append(sub_folder)
print(classes)    

Combine Dataset Sub folders into a single folder and Resize images acording to the model

In [ ]:
X_train=[]
y_train=[]
image_size=224
for i in classes:
    path_train = os.path.join(filename, 'train', i)
    for j in tqdm(os.listdir(path_train)):
        img = cv2.imread(os.path.join(path_train, j))
        img = cv2.resize(img, (image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
    path_test = os.path.join(filename, 'test', i)
    for j in tqdm(os.listdir(path_test)):
        img = cv2.imread(os.path.join(path_test, j))
        img = cv2.resize(img, (image_size, image_size))
        X_train.append(img)
        y_train.append(i)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
print(y_train.shape)

Splitting the dataset into training and testing sets


In [5]:
X_train,y_train = shuffle(X_train, y_train,random_state=42)
lb=LabelEncoder()
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42,stratify=y_train)

Data Augementation of training and Validation sets

In [ ]:

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=7,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True,
     )
test_datagen=ImageDataGenerator(rescale=1./255)
train_datagen.fit(X_train)
test_datagen.fit(X_valid)
print(X_train.shape)
print(X_valid.shape)

In [ ]:
print ("Shape of an image in X_train: ", X_train.shape)
print ("Shape of an image in X_test: ", X_valid.shape)

Encoding Labels(fractured,non_fractured) to integers 0 and 1


In [8]:
labels_train = lb.fit(y_train)
y_train = lb.transform(y_train)
y_valid = lb.transform(y_valid)

In [9]:
y_train = np.array(y_train)
X_train = np.array(X_train)
y_test = np.array(y_valid)
X_test = np.array(X_valid) 

In [ ]:
print("X_train Shape: ", X_train.shape) 
print("X_test Shape: ", X_valid.shape)
print("y_train Shape: ", y_train.shape) 
print("y_test Shape: ", y_valid.shape)

Plot of some training images of dataset

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Define a function to display images
def show_images(images, labels):
    # Create a grid of subplots
    fig, axes = plt.subplots(10, 10, figsize=(9, 9))
    for i, ax in enumerate(axes.flat):
        # Convert BGR image to grayscale
        gray_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        # Display the image in black and white
        ax.imshow(gray_image, cmap='gray')
        # Set the title as the label
        ax.set_title(labels[i])
        # Turn off axis labels
        ax.axis('off')
    # Adjust layout to avoid overlap
    plt.tight_layout()
    # Show the plot
    plt.show()

# Show a sample of images from val_images along with their labels from val_labels
show_images(X_train ,y_train)

Plot of some Validation set images of dataset

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Define a function to display images
def show_images(images, labels):
    # Create a grid of subplots
    fig, axes = plt.subplots(10, 10, figsize=(9, 9))
    for i, ax in enumerate(axes.flat):
        # Convert BGR image to grayscale
        gray_image = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        # Display the image in black and white
        ax.imshow(gray_image, cmap='gray')
        # Set the title as the label
        ax.set_title(labels[i])
        # Turn off axis labels
        ax.axis('off')
    # Adjust layout to avoid overlap
    plt.tight_layout()
    # Show the plot
    plt.show()

# Show a sample of images from val_images along with their labels from val_labels
show_images(X_valid ,y_valid)

Importing Pre-trained VGG16 model

In [13]:
from keras.applications import vgg16

In [14]:
img_rows, img_cols = 224, 224 
vgg = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [15]:
for layer in vgg.layers:
   layer.trainable = False

In [ ]:
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)



Adding Extra Layers over the base VGG16 model

In [23]:
from tensorflow.keras.layers import Dropout
num_classes =2
def lw(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model  = Dense(1024, activation='relu')(top_model)
    top_model = Dropout(0.55)(top_model)
    top_model  = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(1, activation='sigmoid')(top_model)
    return top_model


In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model
FC_Head = lw(vgg, num_classes)
model = Model(inputs = vgg.input, outputs = FC_Head)



Model Summary

In [ ]:
from keras.models import Model
print(model.summary())

Defining Optimizer and model compiling

In [26]:
from tensorflow.keras.models import Model
opt=Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss = 'binary_crossentropy',metrics = ['accuracy'])

In [27]:
early_callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=15, restore_best_weights=True)

Training the model

In [ ]:
history = model.fit(X_train,y_train,
                    epochs=30, 
                    validation_data=(X_test,y_test),
                    #verbose = 1,
                    batch_size=15,
                    callbacks=early_callbacks
                )

Plotting ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
probs = model.predict(X_test).ravel()  # Probabilities of positive class

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, probs)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC Curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

# Calculate AUC score
auc_score = roc_auc_score(y_test, probs)
print("AUC Score:", auc_score)


Accuracy and loss Plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
    # Get the loss values
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Plot the losses with connected lines
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'b-', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Assuming you have the 'history' object from model.fit() method
plot_loss(history)


Validation Accuracy

In [ ]:
# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_test,y_test)

# Print the validation accuracy
print("Validation Accuracy:", val_accuracy)

Saving the model

In [ ]:
from tensorflow.keras.models import load_model

# Assuming you have trained and saved your model as 'model'
model.save('wrist_fracture2.keras')